# Merging Tables
## Datasets
### Collisions
The main dataset. The goal is to add additional information on vehicles involved in each collision as well as weather information. This is done by joining the collisions dataset with the vehicles and weather datasets.

In [125]:
import pandas as pd

In [126]:
collisions = pd.read_csv('Motor_Vehicle_Collisions_cpy2.csv')
collisions.info()

/var/folders/c0/_kktl72s23n7b2y70d5d7ldc0000gn/T/ipykernel_22824/2786592164.py:1: DtypeWarning: Columns (4,40) have mixed types. Specify dtype option on import or set low_memory=False.
  collisions = pd.read_csv('Motor_Vehicle_Collisions_cpy2.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2055607 entries, 0 to 2055606
Data columns (total 41 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Unnamed: 0                     int64  
 1   CRASH DATE                     object 
 2   CRASH TIME                     object 
 3   BOROUGH                        object 
 4   ZIP CODE                       object 
 5   LATITUDE                       float64
 6   LONGITUDE                      float64
 7   LOCATION                       object 
 8   ON STREET NAME                 object 
 9   CROSS STREET NAME              object 
 10  OFF STREET NAME                object 
 11  NUMBER OF PERSONS INJURED      float64
 12  NUMBER OF PERSONS KILLED       float64
 13  NUMBER OF PEDESTRIANS INJURED  int64  
 14  NUMBER OF PEDESTRIANS KILLED   int64  
 15  NUMBER OF CYCLIST INJURED      int64  
 16  NUMBER OF CYCLIST KILLED       int64  
 17  NUMBER OF MOTORIST INJURED     int64  
 18  NU

### Vehicles

In [127]:
vehicles = pd.read_csv('Vehicles_Involved_Cleaned.csv')
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4227359 entries, 0 to 4227358
Data columns (total 18 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   UNIQUE_ID                    int64 
 1   COLLISION_ID                 int64 
 2   CRASH_DATE                   object
 3   VEHICLE_ID                   object
 4   STATE_REGISTRATION           object
 5   VEHICLE_MAKE                 object
 6   VEHICLE_YEAR                 object
 7   TRAVEL_DIRECTION             object
 8   DRIVER_SEX                   object
 9   DRIVER_LICENSE_STATUS        object
 10  PRE_CRASH                    object
 11  POINT_OF_IMPACT              object
 12  VEHICLE_DAMAGE               object
 13  VEHICLE_DAMAGE_1             object
 14  VEHICLE_DAMAGE_2             object
 15  VEHICLE_DAMAGE_3             object
 16  PUBLIC_PROPERTY_DAMAGE       object
 17  PUBLIC_PROPERTY_DAMAGE_TYPE  object
dtypes: int64(2), object(16)
memory usage: 580.5+ MB


Currently the vehicle table is organized such that each row is info on a vehicle involved in a collision. Ideally, we would pivot it to a table such that each row is info on an overall collision with details on every vehicle involved residing on the same row. This would make it easy to merge with the collision table and add additional vehicle info to it. However, it is too memory intensive to pivot the table such that each row is a collision where all the info for each individual vehicle is added as a column. In the case of a rare collision with say 50 vehicles involved, the table would grow to have 50 VEHICLE_MAKE columns and 50 VEHICLE_YEAR columns and so on. Eventually, the table would grow to have hundreds/thousands of columns with millions of rows since there are millions of collisions accounted for. Restricting the table such that it only contains data about collisions with 5 or fewer vehicles can help restrain the size of the pivoted table while also accounting for the vast majority of collisions.


In [128]:
"""
Group rows by COLLISION_ID so vehicles in the same collision are together
Create a column that is the number of rows a particular COLLISION_ID has (i.e. number of vehicles involved in each collision)
Get info on collision numbers
"""
vehicles['GROUP_SIZE'] = vehicles.groupby('COLLISION_ID')['COLLISION_ID'].transform('size')
total_collisions = vehicles['COLLISION_ID'].nunique()
five_or_less = vehicles[vehicles['GROUP_SIZE'] <= 5]['COLLISION_ID'].nunique()
more_than_five = vehicles[vehicles['GROUP_SIZE'] > 5]['COLLISION_ID'].nunique()
print(f'Total number of collisions accounted for: {total_collisions}')
print(f'Number of collisions involving 5 or fewer vehicles: {five_or_less}')
print(f'Number of collisions involving more than 5 vehicles: {more_than_five}')

Total number of collisions accounted for: 2104161
Number of collisions involving 5 or fewer vehicles: 2100774
Number of collisions involving more than 5 vehicles: 3387


In [129]:
"""
Drop the rows where there are more than 5 vehicles involved
Reset the index of the table
"""
vehicles = vehicles[vehicles['GROUP_SIZE'] <= 5].drop(columns=['GROUP_SIZE'])
vehicles.reset_index(drop=True, inplace=True)
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4204448 entries, 0 to 4204447
Data columns (total 18 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   UNIQUE_ID                    int64 
 1   COLLISION_ID                 int64 
 2   CRASH_DATE                   object
 3   VEHICLE_ID                   object
 4   STATE_REGISTRATION           object
 5   VEHICLE_MAKE                 object
 6   VEHICLE_YEAR                 object
 7   TRAVEL_DIRECTION             object
 8   DRIVER_SEX                   object
 9   DRIVER_LICENSE_STATUS        object
 10  PRE_CRASH                    object
 11  POINT_OF_IMPACT              object
 12  VEHICLE_DAMAGE               object
 13  VEHICLE_DAMAGE_1             object
 14  VEHICLE_DAMAGE_2             object
 15  VEHICLE_DAMAGE_3             object
 16  PUBLIC_PROPERTY_DAMAGE       object
 17  PUBLIC_PROPERTY_DAMAGE_TYPE  object
dtypes: int64(2), object(16)
memory usage: 577.4+ MB


In [130]:
for column in vehicles.columns:
    nan_count = vehicles[column].isna().sum()
    print(f'{column}: {nan_count} NaN values')

UNIQUE_ID: 0 NaN values
COLLISION_ID: 0 NaN values
CRASH_DATE: 0 NaN values
VEHICLE_ID: 0 NaN values
STATE_REGISTRATION: 324336 NaN values
VEHICLE_MAKE: 0 NaN values
VEHICLE_YEAR: 0 NaN values
TRAVEL_DIRECTION: 0 NaN values
DRIVER_SEX: 0 NaN values
DRIVER_LICENSE_STATUS: 0 NaN values
PRE_CRASH: 0 NaN values
POINT_OF_IMPACT: 0 NaN values
VEHICLE_DAMAGE: 0 NaN values
VEHICLE_DAMAGE_1: 0 NaN values
VEHICLE_DAMAGE_2: 0 NaN values
VEHICLE_DAMAGE_3: 0 NaN values
PUBLIC_PROPERTY_DAMAGE: 0 NaN values
PUBLIC_PROPERTY_DAMAGE_TYPE: 0 NaN values


22,911 rows of vehicle info were removed after restricting the table to only contain info on vehicles involved in collisions with 5 or fewer vehicles. These rows amount to 3387 collisions being lost, which is small compared to the total of 2104161 collisions.

In [131]:
"""
Pivot the table
Get a vehicle_number column for indexing variations of the same column (i.e. VEHCILE_MAKE will have 5 columns of the same name with indices 1-5 for each vehicle)
The pivot is done such that each row is based on a COLLISION_ID
Flatten the columns such that instead of 5 indices of one column you have 5 columns with the index appended to the end (i.e. VEHICLE_MAKE_1, VEHICLE_MAKE_2, etc)
Reset index
"""
vehicles['vehicle_number'] = vehicles.groupby('COLLISION_ID').cumcount() + 1
vehicles = vehicles.pivot_table(index='COLLISION_ID',
                                columns='vehicle_number',
                                values=['VEHICLE_ID', 'STATE_REGISTRATION', 'VEHICLE_MAKE', 'VEHICLE_YEAR', 'TRAVEL_DIRECTION',
                                        'DRIVER_SEX', 'DRIVER_LICENSE_STATUS', 'PRE_CRASH', 'POINT_OF_IMPACT', 'VEHICLE_DAMAGE',
                                        'VEHICLE_DAMAGE_1', 'VEHICLE_DAMAGE_2', 'VEHICLE_DAMAGE_3', 'PUBLIC_PROPERTY_DAMAGE',
                                        'PUBLIC_PROPERTY_DAMAGE_TYPE'],
                                aggfunc='first')
flattened_columns = []
for col in vehicles.columns:
    flat_col = '_'.join(map(str, col))
    flattened_columns.append(flat_col)

vehicles.columns = flattened_columns
vehicles.reset_index(inplace=True)
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100774 entries, 0 to 2100773
Data columns (total 76 columns):
 #   Column                         Dtype 
---  ------                         ----- 
 0   COLLISION_ID                   int64 
 1   DRIVER_LICENSE_STATUS_1        object
 2   DRIVER_LICENSE_STATUS_2        object
 3   DRIVER_LICENSE_STATUS_3        object
 4   DRIVER_LICENSE_STATUS_4        object
 5   DRIVER_LICENSE_STATUS_5        object
 6   DRIVER_SEX_1                   object
 7   DRIVER_SEX_2                   object
 8   DRIVER_SEX_3                   object
 9   DRIVER_SEX_4                   object
 10  DRIVER_SEX_5                   object
 11  POINT_OF_IMPACT_1              object
 12  POINT_OF_IMPACT_2              object
 13  POINT_OF_IMPACT_3              object
 14  POINT_OF_IMPACT_4              object
 15  POINT_OF_IMPACT_5              object
 16  PRE_CRASH_1                    object
 17  PRE_CRASH_2                    object
 18  PRE_CRASH_3           

In [132]:
for column in vehicles.columns:
    nan_count = vehicles[column].isna().sum()
    print(f'{column}: {nan_count} NaN values')

COLLISION_ID: 0 NaN values
DRIVER_LICENSE_STATUS_1: 0 NaN values
DRIVER_LICENSE_STATUS_2: 199589 NaN values
DRIVER_LICENSE_STATUS_3: 1939640 NaN values
DRIVER_LICENSE_STATUS_4: 2066435 NaN values
DRIVER_LICENSE_STATUS_5: 2093758 NaN values
DRIVER_SEX_1: 0 NaN values
DRIVER_SEX_2: 199589 NaN values
DRIVER_SEX_3: 1939640 NaN values
DRIVER_SEX_4: 2066435 NaN values
DRIVER_SEX_5: 2093758 NaN values
POINT_OF_IMPACT_1: 0 NaN values
POINT_OF_IMPACT_2: 199589 NaN values
POINT_OF_IMPACT_3: 1939640 NaN values
POINT_OF_IMPACT_4: 2066435 NaN values
POINT_OF_IMPACT_5: 2093758 NaN values
PRE_CRASH_1: 0 NaN values
PRE_CRASH_2: 199589 NaN values
PRE_CRASH_3: 1939640 NaN values
PRE_CRASH_4: 2066435 NaN values
PRE_CRASH_5: 2093758 NaN values
PUBLIC_PROPERTY_DAMAGE_1: 0 NaN values
PUBLIC_PROPERTY_DAMAGE_2: 199589 NaN values
PUBLIC_PROPERTY_DAMAGE_3: 1939640 NaN values
PUBLIC_PROPERTY_DAMAGE_4: 2066435 NaN values
PUBLIC_PROPERTY_DAMAGE_5: 2093758 NaN values
PUBLIC_PROPERTY_DAMAGE_TYPE_1: 0 NaN values
PUBL

## Merge

### Collisions with Vehicles
We use a left join, with any matching rows vehicles has with collisions having their columns added to the collisions dataset. If any collision rows have no matching vehicles row, then its added columns will just have NaN values.

In [133]:
collisions_vehicles_merged = collisions.merge(vehicles, on='COLLISION_ID', how='left')
collisions_vehicles_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2055607 entries, 0 to 2055606
Columns: 116 entries, Unnamed: 0 to VEHICLE_YEAR_5
dtypes: bool(2), float64(4), int64(9), object(101)
memory usage: 1.7+ GB


In [134]:
for column in collisions_vehicles_merged.columns:
    nan_count = collisions_vehicles_merged[column].isna().sum()
    print(f'{column}: {nan_count} NaN values')

Unnamed: 0: 0 NaN values
CRASH DATE: 0 NaN values
CRASH TIME: 0 NaN values
BOROUGH: 605690 NaN values
ZIP CODE: 605949 NaN values
LATITUDE: 139087 NaN values
LONGITUDE: 139087 NaN values
LOCATION: 193855 NaN values
ON STREET NAME: 419028 NaN values
CROSS STREET NAME: 751466 NaN values
OFF STREET NAME: 1698966 NaN values
NUMBER OF PERSONS INJURED: 0 NaN values
NUMBER OF PERSONS KILLED: 0 NaN values
NUMBER OF PEDESTRIANS INJURED: 0 NaN values
NUMBER OF PEDESTRIANS KILLED: 0 NaN values
NUMBER OF CYCLIST INJURED: 0 NaN values
NUMBER OF CYCLIST KILLED: 0 NaN values
NUMBER OF MOTORIST INJURED: 0 NaN values
NUMBER OF MOTORIST KILLED: 0 NaN values
CONTRIBUTING FACTOR VEHICLE 1: 6899 NaN values
CONTRIBUTING FACTOR VEHICLE 2: 323898 NaN values
CONTRIBUTING FACTOR VEHICLE 3: 1911284 NaN values
CONTRIBUTING FACTOR VEHICLE 4: 2022871 NaN values
CONTRIBUTING FACTOR VEHICLE 5: 2046664 NaN values
COLLISION_ID: 0 NaN values
VEHICLE TYPE CODE 1: 14071 NaN values
VEHICLE TYPE CODE 2: 401142 NaN values
VE

In [135]:
collisions_vehicles_merged.to_csv('Collisions_Vehicles_Merged.csv', index=False)

### Collisions with Vehicles with Weather
Given a collision dataset with weather added on to it by Caleb, we can create a comprehensive table of collisions with additional info on the vehicles involved as well as the weather during the time the collision took place.

In [1]:
import pandas as pd
collisions_weather_merged = pd.read_csv('col_and_weather.csv')
for column in collisions_weather_merged.columns:
    print(column)

NameError: name 'pd' is not defined